In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import tensorflow as tf
from tensorflow.keras import layers
import seaborn as sns

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [44]:
USE_BT = True


DATASET_ROOT="data/"
TRAIN_LOCATIONS_FILE="SignatureLocs_altered.csv"
GENERATED_TRAIN_LOCATIONS_FILE="SignatureLocs_altered_generated.csv"
TEST_LOCATIONS_FILE="TestLocs_altered.csv"
TRAIN_STRENGTHS_FILE="P_Signatures.csv" if USE_BT else "P_SA_Signatures.csv"
GENERATED_TRAIN_STRENGTHS_FILE="P_Signatures_generated.csv" if USE_BT else "P_SA_Signatures_generated.csv"
TEST_STRENGTHS_FILE="P_Tests.csv" if USE_BT else "P_SA_Tests.csv"
NUMBER_OF_BEACONS=57 if USE_BT else 11

In [45]:
df_train_strengths = pd.read_csv(DATASET_ROOT+TRAIN_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_train_locs = pd.read_csv(DATASET_ROOT+TRAIN_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)
df_generated_train_strengths = pd.read_csv(DATASET_ROOT+TRAIN_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_generated_train_locs = pd.read_csv(DATASET_ROOT+TRAIN_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)
df_test_strengths = pd.read_csv(DATASET_ROOT+TEST_STRENGTHS_FILE, sep=';', names=[x for x in range(NUMBER_OF_BEACONS)])
df_test_locs = pd.read_csv(DATASET_ROOT+TEST_LOCATIONS_FILE, sep=';', names=['x','y'], dtype=float)

train_features = df_train_strengths
train_target = df_train_locs
generated_train_features = df_train_strengths
generated_train_target = df_train_locs
test_features = df_test_strengths
test_target = df_test_locs

normalization_values = np.array(train_features)

### Define all models
Currently:
- 3 hidden layer mixed with dropout layers (basic_model) (deep-learn-paper)

In [46]:
def gen_basic_model():
    normalizer = layers.Normalization()
    normalizer.adapt(normalization_values)

    model = tf.keras.models.Sequential([
      normalizer,
      layers.Dense(500, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(500, activation='relu'),
      layers.Dropout(0.5),
      layers.Dense(2, activation='relu'),
      layers.Dropout(0.5)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    loss_function = tf.keras.losses.MeanSquaredError()

    model.compile(optimizer=optimizer,
                    loss=loss_function)
    
    return model


In [47]:
# Define more models

### Train and evaluate all models in the same way

In [48]:
model_generators = [
    (gen_basic_model, 'Basic model, 3 hidden layers (500) with dropout (0.5)')
]
EPOCHS = 30000
BATCH_SIZE = 8


models_default = [m[0]() for m in model_generators]
models_generated = [m[0]() for m in model_generators]

histories = []
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2000)
for i, model in enumerate(models_default):
    print(f"Training model '{model_generators[i][1]}' on original data")
    histories.append(model.fit(train_features, train_target, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es]))

for i, model in enumerate(models_generated):
    print(f"Training model '{model_generators[i][1]}' on generated data")
    histories.append(model.fit(train_features, train_target, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[es]))

Training model Basic model, 3 hidden layers (500) with dropout (0.5) on original data
Epoch 1/10000
16/16 [==============================] - 1s 1ms/step - loss: 544.5614
Epoch 2/10000
16/16 [==============================] - 0s 1ms/step - loss: 382.7759
Epoch 3/10000
16/16 [==============================] - 0s 1ms/step - loss: 372.0691
Epoch 4/10000
16/16 [==============================] - 0s 1ms/step - loss: 389.2515
Epoch 5/10000
16/16 [==============================] - 0s 1ms/step - loss: 384.9088
Epoch 6/10000
16/16 [==============================] - 0s 1ms/step - loss: 366.3115
Epoch 7/10000
16/16 [==============================] - 0s 1ms/step - loss: 374.8130
Epoch 8/10000
16/16 [==============================] - 0s 1ms/step - loss: 335.7382
Epoch 9/10000
16/16 [==============================] - 0s 1ms/step - loss: 317.8051
Epoch 10/10000
16/16 [==============================] - 0s 1ms/step - loss: 371.9595
Epoch 11/10000
16/16 [==============================] - 0s 1ms/step - los

In [49]:
for i, model in enumerate(models_default):
    print(f"Evaluating original data model '{model_generators[i][1]}'")
    model.evaluate(test_features, test_target)

for i, model in enumerate(models_generated):
    print(f"Evaluating generated data model '{model_generators[i][1]}'")
    model.evaluate(test_features, test_target)

Evaluating model Basic model, 3 hidden layers (500) with dropout (0.5) on original data
15/15 [==============================] - 0s 866us/step - loss: 244.0955
Evaluating model Basic model, 3 hidden layers (500) with dropout (0.5) on generated data
15/15 [==============================] - 0s 898us/step - loss: 219.4102
